In [29]:
print("hello huggingface")

hello huggingface


In [4]:
# ! pip install git+https://github.com/huggingface/transformers
! pip install -U sagemaker
#! pip install git+https://github.com/huggingface/transformers.git
! pip install git+https://github.com/huggingface/transformers.git
#! pip install transformers==4.57.1 # currently, V4.57.0 is not released


  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-pp46w569
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-pp46w569
  Resolved https://github.com/huggingface/transformers.git to commit 61cafd991d5692d5637f39348a7ccb1efaf65446
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached huggingface_hub-1.1.4-py3-none-any.whl.metadata (13 kB)
Using cached huggingface_hub-1.1.4-py3-none-any.whl (515 kB)
  Created wheel for transformers: filename=transformers-5.0.0.dev0-py3-none-any.whl size=11073182 sha256=5992e1e36ef82c7805933b04e3a8b74e4016090d9eb098f31eae6cfc7a6cbb33
  Stored in directory: /tmp/pip-ephem-wheel-cache-09n2cczi/wheels/54/cb/3f/83103de5575c534436d6a4686686dead458238dfaf1147e98d
Successfully built transformers
  Attempting uninstall: huggingface-hub
    Found existing installation: hu

In [ ]:
import IPython
IPython.Application.instance().kernel.do_shutdown(True)

{'status': 'ok', 'restart': True}

: 

In [17]:
import json
import sagemaker
import boto3
from sagemaker.huggingface import HuggingFaceModel, get_huggingface_llm_image_uri


In [18]:


try:
	role = sagemaker.get_execution_role()
except ValueError:
	iam = boto3.client('iam')
	role = iam.get_role(RoleName='sagemaker_execution_role')['Role']['Arn']

hub = {
	'HF_MODEL_ID':'Qwen/Qwen2.5-VL-3B-Instruct',
	'SM_NUM_GPUS': json.dumps(1)
}

huggingface_model = HuggingFaceModel(
	image_uri=get_huggingface_llm_image_uri("huggingface",version="3.3.6"),
	env=hub,
	role=role, 
)


In [ ]:
# deploy model to SageMaker Inference
predictor = huggingface_model.deploy(
	initial_instance_count=1,
	instance_type="ml.g5.2xlarge",
	container_startup_health_check_timeout=300,
  )

-------------

In [ ]:
# example request, you always need to define "inputs"
# data = {
#     "messages": [
#         {
#             "role": "user",
#             "content": [
#                 {
#                     "type": "image",
#                     "image": "https://qianwen-res.oss-cn-beijing.aliyuncs.com/Qwen-VL/assets/demo.jpeg",
#                 },
#                 {"type": "text", "text": "Describe this image."},
#             ]
#         }
#     ],
#     "max_new_tokens": 1024,
#     "temperature": 0.7
# }

# # request
# predictor.predict(data)

predictor.predict({
	"inputs": "Hi, what can you help me with?",
})

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:20                                                                                   │
│                                                                                                  │
│   17 }                                                                                           │
│   18                                                                                             │
│   19 # request                                                                                   │
│ ❱ 20 predictor.predict(data)                                                                     │
│   21                                                                                             │
│                                                                                                  │
│ /opt/conda/lib/python3.12/site-packages/sagemaker/base_predictor.py:212 in predict               │
│                                                                                                  │
│   209 │   │   if inference_component_name:                                                       │
│   210 │   │   │   request_args["InferenceComponentName"] = inference_component_name              │
│   211 │   │                                                                                      │
│ ❱ 212 │   │   response = self.sagemaker_session.sagemaker_runtime_client.invoke_endpoint(**req   │
│   213 │   │   return self._handle_response(response)                                             │
│   214 │                                                                                          │
│   215 │   def _handle_response(self, response):                                                  │
│                                                                                                  │
│ /opt/conda/lib/python3.12/site-packages/botocore/client.py:602 in _api_call                      │
│                                                                                                  │
│    599 │   │   │   │   │   f"{py_operation_name}() only accepts keyword arguments."              │
│    600 │   │   │   │   )                                                                         │
│    601 │   │   │   # The "self" in this scope is referring to the BaseClient.                    │
│ ❱  602 │   │   │   return self._make_api_call(operation_name, kwargs)                            │
│    603 │   │                                                                                     │
│    604 │   │   _api_call.__name__ = str(py_operation_name)                                       │
│    605                                                                                           │
│                                                                                                  │
│ /opt/conda/lib/python3.12/site-packages/botocore/context.py:123 in wrapper                       │
│                                                                                                  │
│   120 │   │   │   with start_as_current_context():                                               │
│   121 │   │   │   │   if hook:                                                                   │
│   122 │   │   │   │   │   hook()                                                                 │
│ ❱ 123 │   │   │   │   return func(*args, **kwargs)                                               │
│   124 │   │                                                                                      │
│   125 │   │   return wrapper                                                                     │
│   126                                                                                            │
│                                                                                                  │
│ /opt/conda/lib/python3.12/site-packages/botocore/client.py:

In [ ]:
predictor.delete_model()
predictor.delete_endpoint()